In [1]:
from Acesso import Login
from Query import Query
import pandas as pd
from datetime import datetime,timedelta

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)


In [2]:
querys={
    
    'Vendas':
    
    """
    
    SELECT ped.[Data de Emissão],cli.[Data de Cadastro],
    CASE WHEN ped.[Data de Emissão]=cli.[Data de Cadastro] THEN 'S' ELSE 'N' END AS [Positivado],
    ped.Pedido,ped.[ID Cliente],cli.[Razão Social],cli.[Nome Fantasia],
    ped.[ID Vendedor],vend.[Nome Resumido],vend.DDD,vend.Telefone,sup.Equipe,
    sup.[ID Sup],sup.Supervisor,sup.[DDD Sup],sup.[Telefone Sup],
    sup.[ID Gerente],sup.Gerente,sup.[DDD Gerente],sup.[Telefone Gerente],
    SUM(ped.[Total Venda]) AS [Total Venda]
    FROM netfeira.vw_venda_estatico ped
    INNER JOIN netfeira.vw_cliente cli ON ped.[ID Cliente]=cli.[ID Cliente]
    INNER JOIN netfeira.vw_vendedor vend ON ped.[ID Vendedor]=vend.[ID Vendedor] --AND vend.Categoria='CLT'
    INNER JOIN netfeira.vw_supervisor sup ON vend.[ID Equipe]=sup.[ID Equipe]
    WHERE [Data de Emissão] BETWEEN DATEADD(DAY,1,
    DATEADD(DAY,DAY(DATEADD(DAY,DAY(GETDATE())*-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)))*-1,
    DATEADD(DAY,DAY(GETDATE())*-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)))) AND
    DATEADD(DAY,DAY(GETDATE())*-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101))
    AND [Tipo de Operação]='VENDAS'
    GROUP BY ped.[Data de Emissão],cli.[Data de Cadastro],ped.Pedido,ped.[ID Cliente],cli.[Razão Social],cli.[Nome Fantasia],
    ped.[ID Vendedor],vend.[Nome Resumido],vend.DDD,vend.Telefone,sup.Equipe,
    sup.[ID Sup],sup.Supervisor,sup.[DDD Sup],sup.[Telefone Sup],
    sup.[ID Gerente],sup.Gerente,sup.[DDD Gerente],sup.[Telefone Gerente]    
    
    """
}

In [3]:
df=sql.CriarTabela(kwargs=querys)

In [4]:
df['Lista']=df['Vendas'].groupby(['ID Cliente'],as_index=False).agg({'Total Venda':'sum'})

In [5]:
codigos=df['Lista'].loc[df['Lista']['Total Venda']>0,'ID Cliente'].unique().tolist()

In [6]:
df['Vendas'].columns

Index(['Data de Emissão', 'Data de Cadastro', 'Positivado', 'Pedido',
       'ID Cliente', 'Razão Social', 'Nome Fantasia', 'ID Vendedor',
       'Nome Resumido', 'DDD', 'Telefone', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente', 'Total Venda'],
      dtype='object')

In [7]:
df['Consolidado']=df['Vendas'].loc[(df['Vendas']['ID Cliente'].isin(codigos))&(df['Vendas']['Positivado']=='S')].groupby(['ID Cliente', 'Razão Social', 'Nome Fantasia','Data de Cadastro', 'ID Vendedor',
       'Nome Resumido', 'DDD', 'Telefone', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente'],as_index=False).agg({'Total Venda':'sum'})

In [8]:
df['Consolidado']['Data Mín']=df['Consolidado']['ID Cliente'].apply(lambda info: df['Vendas'].loc[df['Vendas']['ID Cliente']==info,'Data de Emissão'].min())

In [9]:
df['Consolidado']['Pedido']=df['Consolidado']['ID Cliente'].apply(lambda info: df['Vendas'].loc[df['Vendas']['ID Cliente']==info,'Pedido'].min())

In [10]:
df['Temp']=df['Consolidado'].groupby(['ID Vendedor',
       'Nome Resumido', 'DDD', 'Telefone', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente'],as_index=False).agg({'ID Cliente':'count'}).sort_values('ID Cliente',ascending=False)

In [11]:
df['Temp'].rename(columns={'ID Cliente':'Cliente'},inplace=True)

In [12]:
colunas={'ID Sup':'ID Gerente','ID Gerente':'ID Gerente'}

nomes={'ID Sup':'Supervisor','ID Gerente':'Gerente'}

tel_ddd={'ID Sup':'DDD Sup','ID Gerente':'DDD Gerente'}

tel_num={'ID Sup':'Telefone Sup','ID Gerente':'Telefone Gerente'}

meses={1:'JANEIRO',2:'FEVEREIRO',3:'MARÇO',4:'ABRIL',5:'MAIO',6:'JUNHO',7:'JULHO',8:'AGOSTO',9:'SETEMBRO',10:'OUTUBRO',11:'NOVEMBRO',12:'DEZEMBRO'}

for col1,col2 in colunas.items():
    
    codigos=df['Temp'][col1].unique().tolist()
    
    if(len(codigos)<=0):
        
        continue
    
    for c in codigos:
        
        temp_df=pd.DataFrame()
        
        cod=df['Temp'].loc[df['Temp'][col1]==c,col2].values[-1]
        
        if(cod==c and col1=='ID Sup'):
            
            continue        
        
        nome=str(df['Temp'].loc[df['Temp'][col1]==c,nomes[col1]].values[-1]).title()
        
        ddd=df['Temp'].loc[df['Temp'][col1]==c,tel_ddd[col1]].values[-1]
        
        telefone=df['Temp'].loc[df['Temp'][col1]==c,tel_num[col1]].values[-1]
        
        msg='Bom dia' if datetime.now().hour<12 else 'Boa tarde'
        
        dt_ant=datetime.now()-timedelta(days=datetime.now().date().day)
        
        temp_df=df['Temp'].loc[df['Temp'][col1]==c]
        
        clientes=temp_df['Cliente'].sum()
            
        mensagem=f"""
        
        {msg};
        
        {nome} identificamos referente ao mês de {str(meses[dt_ant.month]).title()} de {dt_ant.year}, um total de {clientes} cliente(s) novo(s) na base de dados da empresa.
        
        """
        
        print(mensagem)
       
        pass
    
    
    pass




        
        Bom dia;
        
        Raquel Sobrinho identificamos referente ao mês de Novembro de 2022, um total de 47 cliente(s) novo(s) na base de dados da empresa.
        
        

        
        Bom dia;
        
        Ulisses Bacci identificamos referente ao mês de Novembro de 2022, um total de 5 cliente(s) novo(s) na base de dados da empresa.
        
        

        
        Bom dia;
        
        Carlos Passos identificamos referente ao mês de Novembro de 2022, um total de 3 cliente(s) novo(s) na base de dados da empresa.
        
        

        
        Bom dia;
        
        Douglas Tedeschi identificamos referente ao mês de Novembro de 2022, um total de 1 cliente(s) novo(s) na base de dados da empresa.
        
        

        
        Bom dia;
        
        Rogerio Felipim identificamos referente ao mês de Novembro de 2022, um total de 68 cliente(s) novo(s) na base de dados da empresa.
        
        


In [13]:
df['Consolidado']

,ID Cliente,Razão Social,Nome Fantasia,Data de Cadastro,ID Vendedor,Nome Resumido,DDD,Telefone,Equipe,ID Sup,Supervisor,DDD Sup,Telefone Sup,ID Gerente,Gerente,DDD Gerente,Telefone Gerente,Total Venda,Data Mín,Pedido
0,28908,PABLO OLIVEIRA RODRIGUES ARAUJO 39860387877,PABLO OLIVEIRA RODRI,2022-11-02,JUVILARE,JULIANA SANTOS,11.0,942986187.0,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,0.00,2022-11-02,438537
1,28909,LEANDRO COSTA EVANGELISTA 31266018808,TOCA DO CORUJA BURGE,2022-11-03,JUVILARE,JULIANA SANTOS,11.0,942986187.0,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,886.40,2022-11-03,438756
2,28913,DOUGLAS SILVA DE ARAUJO 27633161825,DOUGLAS SILVA DE ARA,2022-11-03,THAISFRE,THAIS FREITAS,11.0,956291505.0,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,144.24,2022-11-03,438665
3,28914,JESSICA PIRES VECINO DOS PASSOS 22279748851,SEM PALAVRAS DOCES G,2022-11-03,JUVILARE,JULIANA SANTOS,11.0,942986187.0,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,221.60,2022-11-03,438766
4,28915,JET FAST GROUP LTDA,JET FAST,2022-11-03,DINHO,EDUARDO SUDRE,11.0,947449477.0,EQUIPE 4 (VAR),DOUGLASM,DOUGLAS TEDESCHI,11,998664048,ROGERIOF,ROGERIO FELIPIM,11,951283394,333.36,2022-11-03,439075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,29035,NOVA GM COMERCIO DE PRODUTOS ALIMENTICIOS LTDA,NOVA GM COMERCIO,2022-11-29,CELSOCAR,CELSO AMARO,11.0,997293868.0,EQUIPE 1 (REP),ROGERIOF,ROGERIO FELIPIM,11,951283394,ROGERIOF,ROGERIO FELIPIM,11,951283394,273.06,2022-11-29,445182
64,29042,GM SERRALHERIA EM GERAL LTDA,GM SERRALHERIA,2022-11-30,LAINE,ANDRELAINE SOUZA,11.0,942092341.0,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,157.72,2022-11-30,445398
65,29045,ARIANA MARA DA SILVA,ARIANA MARA DA SILVA,2022-11-30,LAINE,ANDRELAINE SOUZA,11.0,942092341.0,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,162.16,2022-11-30,445521
66,29046,SESSENTA MINUTOS ENTRETENIMENTO LTDA,SESSENTA MINUTOS,2022-11-30,LAINE,ANDRELAINE SOUZA,11.0,942092341.0,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,11,942987434,ROGERIOF,ROGERIO FELIPIM,11,951283394,185.13,2022-11-30,445538
